# Phase 3: Data Preparation

## Fannie Mae 2008Q1 Stress Testing - Credit Default Risk Modeling

---

### CRISP-DM Phase 3: Clean, Transform, and Engineer Features

**Goal**: Prepare data with enhanced feature engineering to achieve AUC-ROC > 0.70

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import pickle
import warnings
warnings.filterwarnings('ignore')

RANDOM_STATE = 42
print("Libraries imported successfully!")

## 3.1 Load Data

In [ ]:
# Configuration
DATA_PATH = "2008Q1.csv"
SAMPLE_SIZE = 1000000

# Column Names for Fannie Mae Performance File
COLUMN_NAMES = [
    'loan_sequence_number', 'monthly_reporting_period', 'current_actual_upb',
    'current_loan_delinquency', 'loan_age', 'remaining_months_maturity',
    'repurchase_flag', 'modification_flag', 'zero_balance_code', 'zero_balance_date',
    'current_interest_rate', 'current_deferred_upb', 'due_date_last_paid',
    'mi_recoveries', 'net_sales_proceeds', 'non_mi_recoveries', 'expenses',
    'legal_costs', 'maintenance_costs', 'taxes_insurance_due', 'miscellaneous_expenses',
    'actual_loss_calculation', 'modification_cost', 'step_modification_flag',
    'deferred_payment_mod', 'estimated_ltv', 'zero_balance_removal_upb',
    'delinquent_accrued_interest', 'delinquency_due_disaster', 'borrower_assistance_status'
]

# Add remaining columns for 110 total
for i in range(30, 110):
    COLUMN_NAMES.append(f'col_{i}')

print(f"Loading {SAMPLE_SIZE:,} rows from {DATA_PATH}...")

df = pd.read_csv(
    DATA_PATH,
    sep='|',
    header=None,
    names=COLUMN_NAMES,
    nrows=SAMPLE_SIZE,
    low_memory=False,
    on_bad_lines='skip'
)

print(f"✓ Loaded {len(df):,} records with {df.shape[1]} columns")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

## 3.2 Create Target Variable

In [ ]:
def create_default_flag(status):
    """
    Create binary default indicator:
    - 0: No default (current or minor delinquency 0-2)
    - 1: Default (90+ days delinquent, foreclosure, REO)
    """
    if pd.isna(status):
        return np.nan
    status_str = str(status).strip().upper()
    
    # Default conditions
    if status_str in ['RA', 'XX', 'F', 'R', 'S', 'T', 'N']:
        return 1
    try:
        if int(float(status_str)) >= 3:
            return 1
    except:
        pass
    return 0

print("Creating target variable...")
df['is_default'] = df['current_loan_delinquency'].apply(create_default_flag)

default_dist = df['is_default'].value_counts(dropna=False)
print("\nTarget Distribution:")
print(f"  No Default (0): {default_dist.get(0, 0):,} ({default_dist.get(0, 0)/len(df)*100:.2f}%)")
print(f"  Default (1): {default_dist.get(1, 0):,} ({default_dist.get(1, 0)/len(df)*100:.2f}%)")
print(f"  Missing: {df['is_default'].isna().sum():,}")

## 3.3 Convert Columns to Numeric

In [ ]:
# Define columns to convert to numeric
numeric_cols = [
    'current_actual_upb', 'loan_age', 'remaining_months_maturity',
    'current_interest_rate', 'current_deferred_upb', 'estimated_ltv',
    'delinquent_accrued_interest', 'expenses', 'legal_costs',
    'maintenance_costs', 'taxes_insurance_due'
]

print("Converting columns to numeric...\n")
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        valid_count = df[col].notna().sum()
        print(f"  {col}: {valid_count:,} valid values ({valid_count/len(df)*100:.1f}%)")

## 3.4 Feature Engineering

### Strategy for 70% AUC:
- Create derived features (ratios, flags)
- Handle NaN values properly before creating flags

In [ ]:
print("Creating engineered features...\n")

# 1. UPB to Deferred Ratio (high ratio = higher risk)
# Fill NaN with 0 to avoid NaN in calculation
upb = df['current_actual_upb'].fillna(0)
deferred = df['current_deferred_upb'].fillna(0)
df['upb_deferred_ratio'] = deferred / (upb + 1)
print("  ✓ upb_deferred_ratio")

# 2. Loan Maturity Progress (how far into the loan)
loan_age = df['loan_age'].fillna(0)
remaining = df['remaining_months_maturity'].fillna(360)
df['loan_progress'] = loan_age / (loan_age + remaining + 1)
print("  ✓ loan_progress")

# 3. Interest Rate Stress (higher rates = higher risk during crisis)
interest_rate = df['current_interest_rate'].fillna(df['current_interest_rate'].median())
df['high_interest_flag'] = (interest_rate > 6.5).astype(int)
print("  ✓ high_interest_flag")

# 4. Has Deferred UPB (binary flag)
df['has_deferred_upb'] = (deferred > 0).astype(int)
print("  ✓ has_deferred_upb")

# 5. Modification Flag (loans that were modified)
df['is_modified'] = df['modification_flag'].apply(
    lambda x: 1 if str(x).strip().upper() == 'Y' else 0
)
print("  ✓ is_modified")

# 6. Has Accrued Interest (delinquent interest accumulating)
accrued = df['delinquent_accrued_interest'].fillna(0)
df['has_accrued_interest'] = (accrued > 0).astype(int)
print("  ✓ has_accrued_interest")

# 7. Total Expenses (sum of all expense columns)
expense_cols = ['expenses', 'legal_costs', 'maintenance_costs', 'taxes_insurance_due']
for col in expense_cols:
    if col in df.columns:
        df[col] = df[col].fillna(0)
df['total_expenses'] = df[expense_cols].sum(axis=1)
df['has_expenses'] = (df['total_expenses'] > 0).astype(int)
print("  ✓ total_expenses")
print("  ✓ has_expenses")

# 8. LTV Risk Category
ltv = df['estimated_ltv'].fillna(80)
df['high_ltv_flag'] = (ltv > 80).astype(int)
print("  ✓ high_ltv_flag")

# 9. UPB Risk Categories
df['large_loan_flag'] = (upb > 300000).astype(int)
print("  ✓ large_loan_flag")

# 10. Loan Age Risk (newer loans might be higher risk)
df['new_loan_flag'] = (loan_age < 24).astype(int)
print("  ✓ new_loan_flag")

print("\n✓ All 11 engineered features created!")

## 3.5 Select Features for Modeling

In [ ]:
# Define feature set - 18 features total
all_features = [
    # Original numeric features (8)
    'current_actual_upb',
    'loan_age',
    'remaining_months_maturity', 
    'current_interest_rate',
    'current_deferred_upb',
    'estimated_ltv',
    'delinquent_accrued_interest',
    'total_expenses',
    
    # Engineered features (10)
    'upb_deferred_ratio',
    'loan_progress',
    'high_interest_flag',
    'has_deferred_upb',
    'is_modified',
    'has_accrued_interest',
    'has_expenses',
    'high_ltv_flag',
    'large_loan_flag',
    'new_loan_flag'
]

# Check which features are available
available_features = [f for f in all_features if f in df.columns]
print(f"Available features: {len(available_features)} of {len(all_features)}")
print("\nFeatures:")
for i, f in enumerate(available_features, 1):
    print(f"  {i:2d}. {f}")

## 3.6 Create Modeling Dataset

In [ ]:
# Create modeling dataframe
print("Creating modeling dataset...\n")

df_model = df[available_features + ['is_default']].copy()
print(f"Initial size: {len(df_model):,} records")

# Remove rows with missing target
df_model = df_model.dropna(subset=['is_default'])
print(f"After removing missing targets: {len(df_model):,} records")

# Fill missing numeric features with median
print("\nFilling missing values:")
for col in available_features:
    missing = df_model[col].isnull().sum()
    if missing > 0:
        median_val = df_model[col].median()
        if pd.notna(median_val):
            df_model[col] = df_model[col].fillna(median_val)
            print(f"  {col}: filled {missing:,} with median {median_val:.2f}")
        else:
            df_model[col] = df_model[col].fillna(0)
            print(f"  {col}: filled {missing:,} with 0")

# Remove infinite values
df_model = df_model.replace([np.inf, -np.inf], np.nan).dropna()
print(f"\nAfter removing infinite values: {len(df_model):,} records")

## 3.7 Handle Outliers

In [ ]:
# Remove extreme outliers (1st and 99th percentile) for continuous features
print("Removing outliers...\n")
initial_count = len(df_model)

continuous_features = [
    'current_actual_upb', 'loan_age', 'remaining_months_maturity',
    'current_interest_rate', 'current_deferred_upb', 'estimated_ltv',
    'delinquent_accrued_interest', 'total_expenses', 'upb_deferred_ratio'
]

for col in continuous_features:
    if col in df_model.columns and len(df_model) > 0:
        Q1 = df_model[col].quantile(0.01)
        Q99 = df_model[col].quantile(0.99)
        before = len(df_model)
        df_model = df_model[(df_model[col] >= Q1) & (df_model[col] <= Q99)]
        removed = before - len(df_model)
        if removed > 0:
            print(f"  {col}: removed {removed:,} outliers")

print(f"\nTotal outliers removed: {initial_count - len(df_model):,}")
print(f"Final clean dataset: {len(df_model):,} records")

## 3.8 Feature Scaling

In [ ]:
# Prepare features and target
print("Scaling features...\n")

X = df_model[available_features].values
y = df_model['is_default'].values.astype(int)

# Check for any remaining issues
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"X contains NaN: {np.isnan(X).any()}")
print(f"X contains Inf: {np.isinf(X).any()}")

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("\n✓ Features scaled successfully!")
print(f"\nClass distribution:")
print(f"  No Default (0): {(y==0).sum():,} ({(y==0).sum()/len(y)*100:.2f}%)")
print(f"  Default (1): {(y==1).sum():,} ({(y==1).sum()/len(y)*100:.2f}%)")

## 3.9 Train-Test Split

In [ ]:
# Split data with stratification
print("Splitting data into train/test sets...\n")

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, 
    test_size=0.2, 
    random_state=RANDOM_STATE, 
    stratify=y
)

print(f"Training set: {len(X_train):,} samples ({len(X_train)/len(X_scaled)*100:.0f}%)")
print(f"Test set: {len(X_test):,} samples ({len(X_test)/len(X_scaled)*100:.0f}%)")
print(f"\nDefault rate (train): {y_train.mean()*100:.2f}%")
print(f"Default rate (test): {y_test.mean()*100:.2f}%")

## 3.10 Save Prepared Data

In [ ]:
# Save prepared data for Phase 4
prepared_data = {
    'X_train': X_train,
    'X_test': X_test,
    'y_train': y_train,
    'y_test': y_test,
    'features': available_features,
    'scaler': scaler,
    'df_model': df_model
}

with open('phase3_prepared_data.pkl', 'wb') as f:
    pickle.dump(prepared_data, f)

print("✓ Prepared data saved to phase3_prepared_data.pkl")
print("\n" + "="*60)
print("PHASE 3 SUMMARY")
print("="*60)
print(f"Total features: {len(available_features)}")
print(f"Training samples: {len(X_train):,}")
print(f"Test samples: {len(X_test):,}")
print(f"Default rate: {y.mean()*100:.2f}%")
print("="*60)

---
## ✅ Phase 3 Complete!

### Key Improvements for 70% AUC:
- Sample size: 1,000,000 records
- 18 total features (8 original + 10 engineered)
- Proper NaN handling before feature engineering
- Outlier removal (1st-99th percentile)
- StandardScaler for feature normalization

### Next: Phase 4 - Modeling

Run the next notebook: `Phase4_Modeling.ipynb`